In [5]:
import pandas as pd
df = pd.read_csv("movies.csv")

In [6]:
import pandas as pd

try:
    answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding='latin1', sep=';')
    print("File loaded successfully with 'latin1' encoding and comma delimiter.")
except UnicodeDecodeError:
    try:
        answersDataFrane = pd.read_csv("AnkietaOFilmachIMDB.csv", encoding='windows-1252', sep=';')
        print("File loaded successfully with 'windows-1252' encoding and comma delimiter.")
    except Exception as e:
        print(f"An error occurred with encoding or delimiter: {e}")
except FileNotFoundError:
    print("Error: The file 'AnkietaOFilmachIMDB.csv' was not found.")

File loaded successfully with 'latin1' encoding and comma delimiter.


## Data preparation for user data

In [7]:
answersDataFrane.replace(-1, pd.NA).describe(include='all')


,whose_rating,rating_kuby,rating_ali
count,249,50.00000,50
unique,249,NaN,7
top,12 Years a Slave,NaN,8
freq,1,NaN,15
mean,NaN,6.84000,NaN
std,NaN,1.26749,NaN
min,NaN,5.00000,NaN
25%,NaN,6.00000,NaN
50%,NaN,7.00000,NaN
75%,NaN,8.00000,NaN


In [8]:
answersDataFrane.head()

,whose_rating,rating_kuby,rating_ali
0,12 Years a Slave,NaN,NaN
1,1917,NaN,NaN
2,2001: A Space Odyssey,NaN,NaN
3,3 Idiots,NaN,NaN
4,A Beautiful Mind,NaN,NaN


In [9]:
answersDataFrane.dtypes

whose_rating     object
rating_kuby     float64
rating_ali       object
dtype: object

In [10]:
def rating_cleaner(CurrentDataFrame, rating_name):

    if CurrentDataFrame[rating_name].dtype!="int64":

        if CurrentDataFrame[rating_name].dtype!="float64":
            CurrentDataFrame[rating_name] = CurrentDataFrame[rating_name].astype(str).str.strip()
            CurrentDataFrame[rating_name] = pd.to_numeric(CurrentDataFrame[rating_name], errors='coerce')

        CurrentDataFrame = CurrentDataFrame.fillna(-1)    
        if CurrentDataFrame[rating_name].dtype == 'float64':
            CurrentDataFrame[rating_name] = CurrentDataFrame[rating_name].astype('int64')
    return CurrentDataFrame

In [11]:
for col in answersDataFrane.columns[1:]:
    answersDataFrane = rating_cleaner(answersDataFrane, col)

In [12]:
df = df.sort_values(by="name")

In [13]:
for col in answersDataFrane.columns[1:]:
    extracted_col = answersDataFrane[col]
    df = pd.concat([df, extracted_col], axis=1)

## Data preparation for project

### usuniecie czesci kolumn

In [14]:
if "img_link" in df.columns: df.drop(columns="img_link", inplace=True)
if "id" in df.columns: df.drop(columns="id", inplace=True)
if "cast_id" in df.columns: df.drop(columns="cast_id", inplace=True)
if "director_id" in df.columns: df.drop(columns="director_id", inplace=True)
if "writter_id" in df.columns: df.drop(columns="writter_id", inplace=True)

### funkcja do zmiany stringu w listę

In [15]:
def split_if_string(value):
    if isinstance(value, str):
        return value.split(',')
    return value # Return as is if not a string (e.g., already a list or NaN)

In [16]:
df['cast_name'] = df['cast_name'].apply(split_if_string)
df['writter_name'] = df['writter_name'].apply(split_if_string)
df['director_name'] = df['director_name'].apply(split_if_string)
df['genre'] = df['genre'].apply(split_if_string)

In [17]:
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')

In [18]:
bins = [0, 60, 90, 120, 150, 180, float('inf')]
labels = [0, 1, 2, 3, 4, 5]
#labels = ['very short', 'short', 'normal', 'pretty long', 'long', 'very long']


df['duration_category'] = pd.cut(df['duration'], bins=bins, labels=labels, right=False)


In [19]:
df['duration_category'] = pd.to_numeric(df['duration_category'], errors='coerce')

# Przygotowanie pod Drzewo losowe

### usuniecie nazw, oraz rank

In [20]:
if "name" in df.columns: df.drop(columns="name", inplace=True)
if "rank" in df.columns: df.drop(columns="rank", inplace= True)

### function for one-hot encoding

In [21]:
def one_hot_encode (DataFrame, column_names, prefixes):
    dfInWork = DataFrame
    for column_name, prefix in zip(column_names, prefixes):
        if column_name in dfInWork.columns: 
            df_exploded = dfInWork[column_name].explode()
            df_dummies = pd.get_dummies(df_exploded, prefix=prefix)
            df_genres_encoded = df_dummies.groupby(df_dummies.index).max()
            df_final = pd.merge(dfInWork, df_genres_encoded, left_index=True, right_index=True, how='left')
            dfInWork = df_final.drop(column_name, axis=1)
        else:
            print(f"ERROR, column {column_name} non existing")
    return dfInWork


In [22]:
df = one_hot_encode(df, ("genre", "cast_name", "writter_name", "director_name"), prefixes = ("g_", "c_", "w_", "d_") )


In [23]:
certificate_numbers = ([])
certificate_labels = df["certificate"].unique()

i = 0
for unique in certificate_labels:
    certificate_numbers.append(i)
   # certificate_labels.append(unique)
    i+=1

label_to_number_map = dict(zip(certificate_labels, certificate_numbers))

mapping = {
    "certificate": label_to_number_map
}

df.replace(mapping, inplace=True)



C:\Users\kubah\AppData\Local\Temp\ipykernel_33260\3106441032.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(mapping, inplace=True)


In [24]:
df = df.drop(columns="duration")

### pozbycie się nulli

In [25]:
df = df.dropna()

## pozbycie się ciągłości

In [26]:
df["imdb_rating"] *=10

In [27]:

df.replace(mapping, inplace=True)
for i in df.columns:
    if df[i].dtype == 'float64':
        df[i] = df[i].astype('int64')

### pozbycie się rzadkich aktorów

In [28]:
deleted = 0
undeleted = 0

for column in df.columns:
    if df[column].dtype == "bool":
        if df[column].values.sum() < 3:
            df.drop(columns=column, inplace=True)
            deleted += 1
        else:
            undeleted+=1
print(f"undeleted: {undeleted}")
print(f"deleted: {deleted}")

undeleted: 190
deleted: 4277


### discretising year and number of votes

sprawdzenie wartości dla danych lat

In [29]:
bins = [0, 1944, 1960, 1968, 1994, 2007, 2015, float('inf')]
labels = [0, 1, 2, 3, 4, 5, 6]
#labels = ['very short', 'short', 'normal', 'pretty long', 'long', 'very long']


df['year_category'] = pd.cut(df['year'], bins=bins, labels=labels, right=False)

In [30]:
df = df.drop(columns="year")

### to samo, ale z liczbą głosów

od 1740000 są wysokie,
from 1030000 moderate

In [31]:
voting_ranges = [0, 2*10**5, 4*10**5, 7*10**5, 10**6, float('inf')]
labels = [0, 1, 2, 3, 4, 5, 6]
#labels = ['very short', 'short', 'normal', 'pretty long', 'long', 'very long']


df['imbd_votes_category'] = pd.cut(df['imbd_votes'], bins=bins, labels=labels, right=False)
df = df.drop(columns="imbd_votes")

# Split The DataSet

In [32]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.neural_network import MLPClassifier

In [33]:
df.head()

,imdb_rating,certificate,rating_kuby,rating_ali,duration_category,g__Action,g__Adventure,g__Animation,g__Biography,g__Comedy,...,d__Martin Scorsese,d__Pete Docter,d__Peter Jackson,d__Quentin Tarantino,d__Ridley Scott,d__Sergio Leone,d__Stanley Kubrick,d__Steven Spielberg,year_category,imbd_votes_category
4,90,0,-1,-1,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,6
181,81,1,-1,-1,3,False,False,False,True,False,...,False,False,False,False,False,False,False,False,5,6
121,82,2,-1,-1,2,True,False,False,False,False,...,False,False,False,False,False,False,False,False,6,6
88,83,0,-1,-1,3,False,True,False,False,False,...,False,False,False,False,False,False,True,False,3,6
86,84,3,6,9,4,False,False,False,False,True,...,False,False,False,False,False,False,False,False,5,6


Choice of surveyed person

In [34]:
'''chosenRating = "rating_ali"
for col in df.columns:
    if col.startswith("rating_") and col != chosenRating:
        df.drop(columns=col, inplace=True)



df = df[df[chosenRating] != -1]
X = df.drop(chosenRating, axis=1)
y = df[chosenRating]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
'''

'chosenRating = "rating_ali"\nfor col in df.columns:\n    if col.startswith("rating_") and col != chosenRating:\n        df.drop(columns=col, inplace=True)\n\n\n\ndf = df[df[chosenRating] != -1]\nX = df.drop(chosenRating, axis=1)\ny = df[chosenRating]\n\n# Split the data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)\n'

# Train & test the dataset

In [35]:
'''
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
'''

'\nrf = RandomForestClassifier(n_estimators=10)\nrf.fit(X_train, y_train)\ny_pred = rf.predict(X_test)\n'

In [36]:
'''AnkietaOFilmachIMDB.csv

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from collections import Counter # Import Counter for easily counting occurrences
from matplotlib.colors import Normalize # For normalizing counts to colormap range


minimal = y_test.min() if y_test.min() < y_pred.min() else y_pred.min()
maximal = y_test.max() if y_test.max() > y_pred.max() else y_pred.max()

x = range(minimal,maximal+1,1)
y = range(minimal,maximal+1,1)

data_points = list(zip(y_test, y_pred))

point_counts = Counter(data_points)

unique_points = list(point_counts.keys())
counts = np.array(list(point_counts.values()))

min_count = counts.min()
max_count = counts.max()

#('viridis', 'plasma', 'hot', 'magma', 'cool')
cmap = cm.plasma

norm = Normalize(vmin=min_count, vmax=max_count if max_count > min_count else min_count + 1)


plt.figure(figsize=(10, 8)) 

for (test_val, pred_val) in unique_points:
    count = point_counts[(test_val, pred_val)]
    color = cmap(norm(count))
    plt.scatter(test_val, pred_val, color=color, s=50, alpha=0.7)

plt.plot(x, y, color='gray', alpha=0.6, linestyle='--', label='Idealne przewidywanie')

plt.xlabel("Odpowiedź człowieka")
plt.ylabel("Przewidziana odpowiedź przez algorytm")

sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label('Liczba punktów o tej samej pozycji')


chosenRating_name = chosenRating.replace("rating_", "")
plt.title(f"Przewidywanie oceny ankietowanego ( {chosenRating_name} ) wybranych filmów z top250 filmów IMDb")
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()
'''


'AnkietaOFilmachIMDB.csv\n\nimport matplotlib.pyplot as plt\nimport matplotlib.cm as cm\nfrom collections import Counter # Import Counter for easily counting occurrences\nfrom matplotlib.colors import Normalize # For normalizing counts to colormap range\n\n\nminimal = y_test.min() if y_test.min() < y_pred.min() else y_pred.min()\nmaximal = y_test.max() if y_test.max() > y_pred.max() else y_pred.max()\n\nx = range(minimal,maximal+1,1)\ny = range(minimal,maximal+1,1)\n\ndata_points = list(zip(y_test, y_pred))\n\npoint_counts = Counter(data_points)\n\nunique_points = list(point_counts.keys())\ncounts = np.array(list(point_counts.values()))\n\nmin_count = counts.min()\nmax_count = counts.max()\n\n#(\'viridis\', \'plasma\', \'hot\', \'magma\', \'cool\')\ncmap = cm.plasma\n\nnorm = Normalize(vmin=min_count, vmax=max_count if max_count > min_count else min_count + 1)\n\n\nplt.figure(figsize=(10, 8)) \n\nfor (test_val, pred_val) in unique_points:\n    count = point_counts[(test_val, pred_val)]

In [37]:
'''from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 20), random_state=5)


clf.fit(X_train, y_train)'''

"from sklearn.neural_network import MLPClassifier\n\nclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 20), random_state=5)\n\n\nclf.fit(X_train, y_train)"

In [38]:
'''y_pred = clf.predict(X_test)'''

'y_pred = clf.predict(X_test)'

In [39]:
def calculate_average_human_ai_difference(xs, ys):
    AvgDifference = 0
    for (x,y) in zip(xs, ys):
        AvgDifference += abs(x-y)
    AvgDifference /= len(xs)
    return AvgDifference

def test_setups(dataFrame, numberOfIterations, probeSize, predictColumns, neuralNetworkParametersSets):

    globalParameterAverageDifferenceMeanArray = ([])
    globalColumnAverageDifferenceMeanArray = ([])
    
# - - - - - - - - - - - - Calculating for every paramether set - - - - - - - - - - - - 
    for neuralNetworkParametersSet in neuralNetworkParametersSets:
        parameterAverageDifferenceMeanArray = ([])
        # Get the data (ready for sgd and adam)
     #   activation, solver, alpha, learning_rate_init, hidden_layer_sizes, max_iter = neuralNetworkParametersSet
        
        #Mean of average difference for one parameter set
        parametersAverageDifferenceMean = 0

        # Mean of average differences for every one column saved
        columnAverageDifferenceMeanArray = ([])
       # temporaryDataFrame = uneditedDataFrame.copy()
    # - - - - - - - - - - - - Calculating for every user input - - - - - - - - - - - - 
        for predictColumn in predictColumns:
        #    temporaryDataFrame = uneditedDataFrame.copy()
            # Mean of average differences for one column
            columnAverageDifferenceMean = 0   
        # - - - - - - - - - - - - Calculating for every iteration - - - - - - - - - - - - 
            for iteration in range(0, numberOfIterations, 1):
                temporaryDataFrame = dataFrame.copy()


                # Drop every column with "rating_" that isnt the one to be predicted
                # Drop every column with "rating_" that isnt the one to be predicted
                for col in list(temporaryDataFrame.columns):
                    if col.startswith("rating_") and col != predictColumn:
                        # Added: Check if column still exists before attempting to drop
                        if col in temporaryDataFrame.columns:
                            temporaryDataFrame.drop(columns=col, inplace=True)

                # Added: Check if predictColumn exists after dropping other columns
                if predictColumn not in temporaryDataFrame.columns:
                    print(f"Warning: predictColumn '{predictColumn}' not found in temporaryDataFrame after column dropping. Skipping iteration.")
                    continue # Skip this iteration if the column is missing

                # Getting rid of unfilled data (to be used in the future)
                temporaryDataFrame = temporaryDataFrame[temporaryDataFrame[predictColumn] != -1]

                # Added: Check if the DataFrame became empty after filtering
                if temporaryDataFrame.empty:
                    print(f"Warning: temporaryDataFrame became empty after filtering for '{predictColumn}'. Skipping iteration.")
                    continue # Skip this iteration if no data remains


                # Splitting the dataset into train and test
                X = temporaryDataFrame.drop(predictColumn, axis=1)
                y = temporaryDataFrame[predictColumn]
                XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=probeSize)

                # Training the network
                # Training the network
                clf = MLPClassifier(**neuralNetworkParametersSet)
              #  clf = MLPClassifier(activation = activation, solver=solver, alpha=alpha,
               #                     learning_rate_init = learning_rate_init,
                #                    hidden_layer_sizes=hidden_layer_sizes, max_iter = max_iter)
                clf.fit(XTrain, yTrain)

                # Testing the network
                yPred = clf.predict(XTest)
                columnAverageDifferenceMean += calculate_average_human_ai_difference(yTest, yPred)

            # Mean of average difference in current column prediction
            columnAverageDifferenceMean /= numberOfIterations
            # Saving such mean
            columnAverageDifferenceMeanArray.append(columnAverageDifferenceMean)

        # Mean of average difference in current parameter set
        parametersAverageDifferenceMean = np.mean(columnAverageDifferenceMeanArray)
        parameterAverageDifferenceMeanArray.append(parametersAverageDifferenceMean)
        globalColumnAverageDifferenceMeanArray.append(columnAverageDifferenceMeanArray)

    #globalParameterAverageDifferenceMeanArray.append(parametersAverageDifferenceMean)
       # print("Appending")
        globalParameterAverageDifferenceMeanArray.append(parameterAverageDifferenceMeanArray) # Use extend to flatten if parameterAverageDifferenceMeanArray is a list of single values
    return globalParameterAverageDifferenceMeanArray, globalColumnAverageDifferenceMeanArray




In [40]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore all ConvergenceWarnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [41]:
import itertools

predictColumns = ("rating_kuby", "rating_ali")


activationSet = ("relu", "tanh")
solverSet = ("sgd", "adam")
alphaSet = (0.0001, 0.001, 0.01, 0.05, 0.1)
learning_rate = ("adaptive", "constant")
learning_rate_initSet =  (0.001, 0.005,)
hidden_layer_sizesSet = ((2,32), (4, 32), (6,32), (4,64))
max_iterSet = (300, 350, 400)

# Generate all combinations
neuralNetworkParametersSets = []
for activation, solver, alpha, learning_rate, learning_rate_init, hidden_layer_sizes, max_iter in itertools.product(
    activationSet,
    solverSet,
    alphaSet,
    learning_rate,
    learning_rate_initSet,
    hidden_layer_sizesSet,
    max_iterSet
):
    param_set = {
        "activation": activation,
        "solver": solver,
        "alpha": alpha,
        "learning_rate": learning_rate,
        "learning_rate_init": learning_rate_init,
        "hidden_layer_sizes": hidden_layer_sizes,
        "max_iter": max_iter
    }
    neuralNetworkParametersSets.append(param_set)

print(f"DEBUG: Number of neuralNetworkParametersSets generated: {len(neuralNetworkParametersSets)}")
print(f"DEBUG: Contents of neuralNetworkParametersSets: {neuralNetworkParametersSets}")

parameter, _ = test_setups(dataFrame = df, numberOfIterations = 7, probeSize = 0.25, predictColumns=predictColumns,neuralNetworkParametersSets=neuralNetworkParametersSets)
print(parameter)

DEBUG: Number of neuralNetworkParametersSets generated: 960
DEBUG: Contents of neuralNetworkParametersSets: [{'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate

In [63]:

tab_values = [[1.1978021978021975], [1.2692307692307692], [1.1428571428571428], [1.2252747252747254], [1.1538461538461537], [1.1868131868131866], [1.1208791208791211], [1.2362637362637363], [1.2417582417582418], [1.1428571428571428], [1.0934065934065935], [1.0549450549450547], [1.1153846153846154], [1.1648351648351647], [1.1538461538461537], [1.10989010989011], [1.192307692307692], [1.1813186813186813], [1.1318681318681318], [1.1593406593406592], [1.2637362637362637], [1.0824175824175821], [1.1318681318681318], [1.1043956043956045], [1.065934065934066], [1.2472527472527473], [1.0714285714285714], [1.3296703296703298], [1.346153846153846], [1.2692307692307692], [1.3406593406593408], [1.0879120879120878], [1.1373626373626373], [1.1483516483516483], [1.142857142857143], [1.1978021978021978], [1.0604395604395604], [1.1263736263736264], [1.1758241758241756], [1.0824175824175823], [1.0494505494505493], [1.2142857142857144], [1.1263736263736264], [1.1373626373626373], [1.1868131868131868], [1.1813186813186811], [1.120879120879121], [1.1868131868131868], [1.197802197802198], [1.1318681318681318], [1.1428571428571428], [1.2472527472527473], [1.2637362637362637], [1.2417582417582418], [1.1813186813186813], [1.1703296703296702], [1.1648351648351647], [1.1758241758241756], [1.10989010989011], [1.2527472527472527], [1.10989010989011], [1.1648351648351647], [1.2472527472527473], [1.263736263736264], [1.1813186813186811], [1.1373626373626373], [1.098901098901099], [1.2252747252747254], [1.2692307692307692], [1.1043956043956042], [1.1648351648351647], [1.0934065934065933], [1.1263736263736264], [1.10989010989011], [1.2032967032967035], [1.346153846153846], [1.2417582417582416], [1.2197802197802197], [1.1923076923076923], [1.1538461538461537], [1.0714285714285716], [1.120879120879121], [1.1153846153846154], [1.1648351648351647], [1.2747252747252746], [1.0714285714285712], [1.0769230769230769], [1.3241758241758244], [1.1318681318681318], [1.197802197802198], [1.2142857142857144], [1.1703296703296702], [1.2417582417582418], [1.0439560439560438], [1.2087912087912085], [1.054945054945055], [1.2252747252747254], [1.3186813186813187], [1.10989010989011], [1.1098901098901097], [1.1373626373626373], [1.2472527472527473], [1.2362637362637363], [1.0384615384615383], [1.2582417582417582], [1.065934065934066], [1.2637362637362637], [1.120879120879121], [1.0934065934065933], [1.0879120879120878], [1.0934065934065935], [1.120879120879121], [1.1483516483516483], [1.1263736263736264], [1.120879120879121], [1.1263736263736264], [1.1428571428571428], [1.0604395604395604], [1.0274725274725274], [1.0769230769230769], [1.0769230769230769], [1.302197802197802], [1.2142857142857142], [1.1978021978021975], [1.412087912087912], [1.3681318681318682], [1.3186813186813187], [1.2252747252747254], [1.2252747252747254], [1.0769230769230766], [1.2307692307692308], [1.2362637362637363], [1.1758241758241756], [1.0769230769230769], [1.1868131868131868], [1.1813186813186816], [1.2417582417582418], [1.1263736263736261], [1.1263736263736264], [1.010989010989011], [1.1043956043956045], [1.208791208791209], [1.291208791208791], [1.0714285714285714], [1.2747252747252749], [1.3131868131868134], [1.1043956043956045], [1.1703296703296702], [1.2197802197802199], [1.1428571428571428], [1.1373626373626373], [1.3241758241758244], [1.214285714285714], [1.1593406593406592], [1.2802197802197801], [1.0824175824175823], [1.0384615384615383], [1.1758241758241759], [1.1593406593406592], [1.0769230769230769], [1.1758241758241756], [1.1703296703296706], [1.225274725274725], [1.225274725274725], [1.1043956043956045], [1.1758241758241756], [1.1428571428571426], [1.120879120879121], [1.1978021978021975], [1.2252747252747251], [1.186813186813187], [1.0824175824175826], [1.1868131868131868], [1.3351648351648353], [1.2417582417582418], [1.2582417582417582], [1.1043956043956042], [1.2802197802197801], [1.2692307692307692], [1.1703296703296702], [1.1648351648351647], [1.1043956043956045], [1.1703296703296702], [1.087912087912088], [1.1428571428571428], [1.1043956043956045], [1.2472527472527473], [1.2747252747252746], [1.2362637362637363], [1.120879120879121], [1.2417582417582418], [1.2252747252747254], [1.2142857142857142], [1.2692307692307692], [1.3461538461538463], [1.0934065934065933], [1.1538461538461537], [1.2802197802197801], [1.1593406593406592], [1.1813186813186811], [1.2197802197802199], [1.1538461538461537], [1.10989010989011], [1.120879120879121], [1.2197802197802197], [1.120879120879121], [1.1263736263736264], [1.2637362637362637], [1.1373626373626373], [1.1153846153846154], [1.1428571428571428], [1.0439560439560438], [1.3076923076923075], [1.0054945054945055], [1.0604395604395604], [1.1373626373626373], [1.2417582417582418], [1.2087912087912085], [1.1483516483516483], [1.0494505494505493], [1.2197802197802197], [1.3351648351648353], [1.175824175824176], [1.1318681318681318], [1.2417582417582418], [1.3021978021978022], [1.2747252747252749], [1.2637362637362637], [1.054945054945055], [1.0604395604395604], [1.0549450549450547], [1.1318681318681318], [1.2032967032967035], [1.1428571428571428], [1.1648351648351647], [1.120879120879121], [1.10989010989011], [1.1868131868131866], [1.2527472527472527], [1.192307692307692], [1.197802197802198], [1.1813186813186811], [1.263736263736264], [1.2252747252747254], [1.2582417582417582], [1.3296703296703294], [1.3846153846153846], [1.3186813186813189], [1.3186813186813189], [1.2637362637362637], [1.2967032967032968], [1.3461538461538463], [1.2252747252747251], [1.3186813186813189], [1.4615384615384615], [1.4285714285714284], [1.467032967032967], [1.3296703296703298], [1.5439560439560438], [1.5934065934065933], [1.521978021978022], [1.4670329670329672], [1.4560439560439562], [1.521978021978022], [1.0934065934065933], [1.3626373626373627], [1.1483516483516485], [1.2032967032967035], [1.208791208791209], [1.3406593406593408], [1.3681318681318682], [1.1538461538461537], [1.3516483516483517], [1.2967032967032968], [1.3296703296703294], [1.2912087912087915], [1.3571428571428572], [1.5439560439560438], [1.39010989010989], [1.3461538461538463], [1.5054945054945055], [1.423076923076923], [1.5494505494505495], [1.5054945054945055], [1.521978021978022], [1.6813186813186816], [1.5604395604395604], [1.489010989010989], [1.2582417582417584], [1.2362637362637363], [1.1923076923076923], [1.4175824175824174], [1.2857142857142856], [1.2472527472527473], [1.186813186813187], [1.2362637362637363], [1.3516483516483517], [1.2857142857142856], [1.4065934065934067], [1.4615384615384617], [1.4835164835164836], [1.4670329670329672], [1.291208791208791], [1.5], [1.467032967032967], [1.4560439560439562], [1.4835164835164836], [1.598901098901099], [1.587912087912088], [1.4450549450549453], [1.3956043956043955], [1.5989010989010988], [1.2307692307692308], [1.2197802197802194], [1.1813186813186813], [1.2252747252747254], [1.2967032967032968], [1.4560439560439562], [1.4230769230769231], [1.2637362637362637], [1.3571428571428572], [1.2362637362637363], [1.3516483516483517], [1.3241758241758241], [1.3076923076923077], [1.3736263736263736], [1.5494505494505495], [1.543956043956044], [1.5329670329670328], [1.4120879120879122], [1.6263736263736264], [1.4505494505494507], [1.6538461538461537], [1.5384615384615383], [1.4065934065934065], [1.3681318681318682], [1.3241758241758241], [1.2692307692307692], [1.3076923076923077], [1.2362637362637363], [1.3516483516483517], [1.2912087912087913], [1.3626373626373625], [1.3076923076923077], [1.4010989010989012], [1.2637362637362637], [1.4175824175824174], [1.39010989010989], [1.4175824175824177], [1.521978021978022], [1.5164835164835164], [1.5659340659340661], [1.3846153846153846], [1.2967032967032965], [1.4340659340659339], [1.521978021978022], [1.4670329670329672], [1.4340659340659343], [1.510989010989011], [1.6428571428571428], [1.2747252747252746], [1.379120879120879], [1.2857142857142856], [1.2252747252747251], [1.3131868131868134], [1.5274725274725274], [1.3956043956043955], [1.379120879120879], [1.434065934065934], [1.1703296703296702], [1.3846153846153846], [1.3681318681318682], [1.4340659340659343], [1.5934065934065935], [1.4615384615384617], [1.4505494505494507], [1.587912087912088], [1.5439560439560438], [1.4450549450549453], [1.3681318681318682], [1.478021978021978], [1.543956043956044], [1.4230769230769231], [1.4835164835164836], [1.065934065934066], [1.1868131868131868], [1.2307692307692306], [1.3351648351648353], [1.2857142857142858], [1.2692307692307692], [1.3571428571428572], [1.3186813186813184], [1.5054945054945055], [1.2857142857142856], [1.2582417582417582], [1.39010989010989], [1.4175824175824174], [1.5549450549450547], [1.4175824175824177], [1.5], [1.4065934065934067], [1.5439560439560438], [1.4505494505494505], [1.4835164835164836], [1.6263736263736264], [1.4065934065934067], [1.4505494505494505], [1.5054945054945055], [1.2087912087912087], [1.2472527472527473], [1.3571428571428572], [1.2307692307692306], [1.2692307692307692], [1.2417582417582418], [1.2967032967032965], [1.3461538461538463], [1.3516483516483517], [1.3021978021978025], [1.302197802197802], [1.4670329670329672], [1.4560439560439558], [1.554945054945055], [1.653846153846154], [1.4670329670329672], [1.4945054945054945], [1.4780219780219779], [1.5439560439560438], [1.478021978021978], [1.6483516483516483], [1.60989010989011], [1.5714285714285714], [1.5054945054945055], [1.2472527472527473], [1.1318681318681318], [1.3516483516483517], [1.2912087912087913], [1.302197802197802], [1.4395604395604396], [1.2307692307692308], [1.2857142857142856], [1.4230769230769234], [1.2912087912087913], [1.3736263736263736], [1.3351648351648353], [1.3461538461538463], [1.2472527472527473], [1.5054945054945055], [1.6318681318681318], [1.489010989010989], [1.4560439560439562], [1.5384615384615383], [1.4285714285714286], [1.6758241758241756], [1.4725274725274726], [1.5384615384615383], [1.5384615384615383], [1.1813186813186811], [1.2417582417582418], [1.3021978021978022], [1.39010989010989], [1.3076923076923075], [1.2967032967032968], [1.3131868131868134], [1.3681318681318682], [1.263736263736264], [1.2967032967032968], [1.3296703296703298], [1.3186813186813184], [1.6263736263736264], [1.4670329670329672], [1.3956043956043955], [1.4395604395604396], [1.489010989010989], [1.4725274725274726], [1.3626373626373627], [1.5164835164835164], [1.6153846153846154], [1.6538461538461537], [1.5164835164835164], [1.456043956043956], [1.054945054945055], [1.0329670329670328], [1.2472527472527473], [1.0879120879120878], [1.1263736263736264], [1.1318681318681318], [1.2637362637362637], [1.10989010989011], [1.291208791208791], [1.192307692307692], [1.0494505494505495], [1.1648351648351647], [1.1098901098901095], [1.0934065934065935], [1.120879120879121], [1.0549450549450547], [1.0989010989010988], [1.1593406593406592], [1.214285714285714], [1.1868131868131866], [1.098901098901099], [1.1428571428571428], [1.0549450549450547], [1.1263736263736264], [1.1593406593406592], [1.1263736263736264], [1.0549450549450547], [1.1043956043956042], [1.1043956043956042], [1.0824175824175823], [1.1098901098901097], [1.1483516483516483], [1.0604395604395604], [1.225274725274725], [1.0604395604395607], [1.098901098901099], [1.0769230769230769], [1.214285714285714], [1.1593406593406592], [1.120879120879121], [1.1318681318681318], [1.0384615384615383], [1.0164835164835164], [1.0714285714285712], [1.098901098901099], [1.1813186813186813], [1.1703296703296702], [1.1153846153846154], [1.1483516483516483], [1.1648351648351647], [1.1153846153846154], [1.2527472527472527], [1.0824175824175823], [1.1428571428571428], [1.098901098901099], [1.2692307692307694], [1.1923076923076923], [1.0769230769230769], [1.1813186813186811], [1.0714285714285716], [1.0659340659340661], [1.098901098901099], [1.0164835164835164], [1.120879120879121], [1.2472527472527473], [1.1153846153846154], [1.1043956043956045], [1.1153846153846154], [1.1593406593406592], [1.1868131868131866], [1.10989010989011], [1.2912087912087913], [1.0494505494505495], [1.0824175824175823], [1.1813186813186813], [1.2637362637362637], [1.0439560439560438], [1.10989010989011], [1.1263736263736264], [1.120879120879121], [1.1538461538461537], [1.1263736263736264], [1.1593406593406592], [1.1483516483516483], [1.120879120879121], [1.1703296703296702], [1.1263736263736264], [1.010989010989011], [1.0714285714285714], [1.0879120879120878], [1.1428571428571428], [1.192307692307692], [1.0769230769230769], [1.1538461538461537], [1.0439560439560438], [1.0164835164835164], [1.076923076923077], [1.1428571428571428], [1.208791208791209], [1.065934065934066], [1.0714285714285716], [1.120879120879121], [1.0769230769230769], [1.2032967032967032], [1.1593406593406594], [1.1538461538461537], [1.1758241758241756], [1.2527472527472527], [1.0714285714285714], [1.175824175824176], [1.1428571428571428], [1.1153846153846154], [1.0549450549450547], [1.0604395604395604], [1.2252747252747254], [1.1593406593406594], [1.0439560439560438], [1.021978021978022], [1.1318681318681318], [1.0549450549450547], [1.1538461538461537], [1.0824175824175823], [1.1703296703296702], [1.1648351648351647], [1.1593406593406592], [1.0604395604395602], [1.1373626373626373], [1.1813186813186811], [1.1373626373626373], [1.087912087912088], [1.1978021978021975], [1.0934065934065933], [1.021978021978022], [1.043956043956044], [1.054945054945055], [1.1813186813186811], [1.2142857142857144], [1.10989010989011], [1.192307692307692], [1.098901098901099], [1.0989010989010988], [1.0164835164835164], [1.1978021978021978], [1.120879120879121], [1.0934065934065935], [1.2032967032967032], [1.054945054945055], [1.1758241758241756], [1.1648351648351647], [1.225274725274725], [1.120879120879121], [1.0659340659340657], [1.1373626373626373], [1.0824175824175826], [1.0934065934065935], [1.098901098901099], [1.1373626373626373], [1.1318681318681318], [0.9945054945054945], [1.1648351648351647], [1.1483516483516483], [1.10989010989011], [1.1703296703296702], [1.2307692307692308], [1.1868131868131866], [1.0494505494505493], [1.1373626373626373], [1.0824175824175823], [1.1043956043956042], [1.2417582417582418], [1.1263736263736264], [1.087912087912088], [1.1593406593406592], [1.1483516483516483], [1.197802197802198], [1.1098901098901097], [1.1428571428571428], [1.1263736263736264], [1.0824175824175826], [1.1978021978021975], [1.1318681318681318], [1.1263736263736266], [1.10989010989011], [1.1483516483516483], [1.0934065934065935], [1.1043956043956045], [1.1813186813186811], [1.1318681318681318], [1.0714285714285714], [1.065934065934066], [1.1538461538461537], [0.9945054945054945], [1.1318681318681318], [1.1538461538461537], [1.1978021978021975], [1.0934065934065935], [1.1318681318681318], [1.1263736263736264], [1.0934065934065935], [1.0604395604395604], [1.120879120879121], [1.120879120879121], [1.1098901098901097], [1.0934065934065933], [1.1813186813186811], [1.1593406593406594], [1.0769230769230769], [1.1153846153846154], [1.0769230769230769], [1.186813186813187], [1.0934065934065935], [1.1538461538461537], [1.1813186813186811], [1.0329670329670328], [1.1318681318681318], [1.1153846153846154], [1.1483516483516483], [1.098901098901099], [1.1483516483516483], [1.175824175824176], [1.120879120879121], [1.1538461538461537], [1.1593406593406592], [1.208791208791209], [1.0769230769230769], [1.1153846153846154], [1.2252747252747251], [1.208791208791209], [1.1373626373626373], [1.1978021978021978], [1.0879120879120878], [1.291208791208791], [1.2472527472527473], [1.076923076923077], [1.0879120879120878], [1.1153846153846154], [1.1483516483516483], [1.043956043956044], [1.1703296703296702], [1.0934065934065935], [1.3241758241758244], [1.4285714285714284], [1.3571428571428572], [1.4120879120879122], [1.4065934065934067], [1.3681318681318682], [1.412087912087912], [1.401098901098901], [1.620879120879121], [1.4010989010989015], [1.2527472527472527], [1.4395604395604398], [1.2912087912087913], [1.39010989010989], [1.5824175824175826], [1.5], [1.6483516483516483], [1.5659340659340657], [1.521978021978022], [1.5], [1.576923076923077], [1.6153846153846154], [1.6263736263736264], [1.4670329670329672], [1.291208791208791], [1.1648351648351647], [1.4285714285714288], [1.4945054945054945], [1.5329670329670328], [1.4120879120879122], [1.642857142857143], [1.5714285714285712], [1.3406593406593408], [1.1483516483516483], [1.554945054945055], [1.4615384615384617], [1.3846153846153846], [1.4230769230769231], [1.5054945054945055], [1.4285714285714284], [1.521978021978022], [1.5054945054945055], [1.5604395604395602], [1.4670329670329672], [1.6868131868131866], [1.5714285714285712], [1.5109890109890112], [1.4945054945054945], [1.4615384615384617], [1.3956043956043958], [1.3736263736263736], [1.4835164835164836], [1.3571428571428572], [1.5054945054945055], [1.5934065934065935], [1.3186813186813187], [1.5879120879120878], [1.510989010989011], [1.4670329670329672], [1.3186813186813187], [1.2692307692307692], [1.4835164835164836], [1.5329670329670328], [1.5714285714285714], [1.4065934065934067], [1.5329670329670328], [1.445054945054945], [1.510989010989011], [1.697802197802198], [1.510989010989011], [1.3901098901098903], [1.5824175824175823], [1.3516483516483517], [1.4065934065934067], [1.2582417582417582], [1.4835164835164836], [1.4230769230769231], [1.3956043956043955], [1.4395604395604398], [1.5769230769230769], [1.598901098901099], [1.291208791208791], [1.5054945054945055], [1.3626373626373627], [1.3626373626373627], [1.3736263736263736], [1.3681318681318682], [1.5549450549450547], [1.5054945054945055], [1.5329670329670328], [1.6318681318681318], [1.401098901098901], [1.5494505494505493], [1.4285714285714286], [1.5], [1.708791208791209], [1.335164835164835], [1.4175824175824174], [1.6263736263736264], [1.3736263736263736], [1.39010989010989], [1.4285714285714288], [1.2087912087912085], [1.478021978021978], [1.510989010989011], [1.4010989010989012], [1.4560439560439562], [1.5824175824175823], [1.3681318681318682], [1.6428571428571428], [1.2692307692307692], [1.510989010989011], [1.4560439560439562], [1.5054945054945055], [1.5274725274725274], [1.5054945054945055], [1.6373626373626373], [1.3736263736263736], [1.5494505494505493], [1.5054945054945055], [1.4505494505494505], [1.4175824175824177], [1.5164835164835164], [1.5934065934065935], [1.478021978021978], [1.434065934065934], [1.406593406593407], [1.5054945054945057], [1.401098901098901], [1.3956043956043955], [1.39010989010989], [1.697802197802198], [1.554945054945055], [1.434065934065934], [1.4175824175824174], [1.489010989010989], [1.263736263736264], [1.5329670329670328], [1.5439560439560438], [1.565934065934066], [1.5329670329670328], [1.478021978021978], [1.565934065934066], [1.5494505494505493], [1.4175824175824177], [1.2417582417582418], [1.2912087912087913], [1.5714285714285714], [1.3351648351648353], [1.4175824175824179], [1.3681318681318682], [1.3021978021978022], [1.5769230769230769], [1.4065934065934067], [1.4175824175824174], [1.4120879120879122], [1.4065934065934065], [1.5549450549450547], [1.6043956043956045], [1.510989010989011], [1.467032967032967], [1.5934065934065935], [1.5384615384615385], [1.532967032967033], [1.5439560439560442], [1.6758241758241756], [1.5824175824175823], [1.5934065934065935], [1.2362637362637363], [1.4505494505494505], [1.3681318681318682], [1.4285714285714286], [1.489010989010989], [1.3681318681318682], [1.3461538461538463], [1.5], [1.5384615384615383], [1.4670329670329672], [1.3296703296703298], [1.4505494505494505], [1.5274725274725274], [1.5329670329670328], [1.4725274725274726], [1.3736263736263736], [1.4835164835164836], [1.4395604395604398], [1.4065934065934065], [1.620879120879121], [1.6098901098901097], [1.478021978021978], [1.4120879120879122], [1.5494505494505493], [1.3846153846153846], [1.5439560439560438], [1.3681318681318682], [1.401098901098901], [1.4890109890109893], [1.5494505494505495], [1.456043956043956], [1.4945054945054945], [1.3461538461538463], [1.2307692307692306], [1.4175824175824177], [1.4835164835164836], [1.3956043956043958], [1.576923076923077], [1.456043956043956], [1.5549450549450552], [1.620879120879121], [1.489010989010989], [1.4945054945054945], [1.6153846153846154], [1.521978021978022], [1.5054945054945055], [1.6043956043956045], [1.5494505494505495], [1.3626373626373627], [1.3076923076923077], [1.4505494505494505], [1.5494505494505495], [1.478021978021978], [1.478021978021978], [1.5604395604395604], [1.4230769230769231], [1.4505494505494505], [1.4560439560439562], [1.4835164835164836], [1.3186813186813189], [1.6318681318681318], [1.5494505494505493], [1.6428571428571428], [1.4395604395604396], [1.708791208791209], [1.4560439560439562], [1.5934065934065933], [1.4395604395604396], [1.6428571428571428], [1.5659340659340657], [1.6208791208791207], [1.521978021978022]]
tab_labels = [{'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}]

setValue = 1.05
lowEnough = []

for (x, label) in zip(tab_values, tab_labels):
    if x[0] < setValue:
        lowEnough.append((x[0], label))  # Extract the float from the list

# Sort by value in descending order
lowEnough.sort(reverse=False, key=lambda pair: pair[0])

# Print results
for val, label in lowEnough:
    print(f"Value: {val} for {label}")

Value: 0.9945054945054945 for {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}
Value: 0.9945054945054945 for {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}
Value: 1.0054945054945055 for {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}
Value: 1.010989010989011 for {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}
Value: 1.010989010989011 for {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}
Value: 1.0164835164835164 for {'activation': 't

In [ ]:

tab_values = [[1.1978021978021975], [1.2692307692307692], [1.1428571428571428], [1.2252747252747254], [1.1538461538461537], [1.1868131868131866], [1.1208791208791211], [1.2362637362637363], [1.2417582417582418], [1.1428571428571428], [1.0934065934065935], [1.0549450549450547], [1.1153846153846154], [1.1648351648351647], [1.1538461538461537], [1.10989010989011], [1.192307692307692], [1.1813186813186813], [1.1318681318681318], [1.1593406593406592], [1.2637362637362637], [1.0824175824175821], [1.1318681318681318], [1.1043956043956045], [1.065934065934066], [1.2472527472527473], [1.0714285714285714], [1.3296703296703298], [1.346153846153846], [1.2692307692307692], [1.3406593406593408], [1.0879120879120878], [1.1373626373626373], [1.1483516483516483], [1.142857142857143], [1.1978021978021978], [1.0604395604395604], [1.1263736263736264], [1.1758241758241756], [1.0824175824175823], [1.0494505494505493], [1.2142857142857144], [1.1263736263736264], [1.1373626373626373], [1.1868131868131868], [1.1813186813186811], [1.120879120879121], [1.1868131868131868], [1.197802197802198], [1.1318681318681318], [1.1428571428571428], [1.2472527472527473], [1.2637362637362637], [1.2417582417582418], [1.1813186813186813], [1.1703296703296702], [1.1648351648351647], [1.1758241758241756], [1.10989010989011], [1.2527472527472527], [1.10989010989011], [1.1648351648351647], [1.2472527472527473], [1.263736263736264], [1.1813186813186811], [1.1373626373626373], [1.098901098901099], [1.2252747252747254], [1.2692307692307692], [1.1043956043956042], [1.1648351648351647], [1.0934065934065933], [1.1263736263736264], [1.10989010989011], [1.2032967032967035], [1.346153846153846], [1.2417582417582416], [1.2197802197802197], [1.1923076923076923], [1.1538461538461537], [1.0714285714285716], [1.120879120879121], [1.1153846153846154], [1.1648351648351647], [1.2747252747252746], [1.0714285714285712], [1.0769230769230769], [1.3241758241758244], [1.1318681318681318], [1.197802197802198], [1.2142857142857144], [1.1703296703296702], [1.2417582417582418], [1.0439560439560438], [1.2087912087912085], [1.054945054945055], [1.2252747252747254], [1.3186813186813187], [1.10989010989011], [1.1098901098901097], [1.1373626373626373], [1.2472527472527473], [1.2362637362637363], [1.0384615384615383], [1.2582417582417582], [1.065934065934066], [1.2637362637362637], [1.120879120879121], [1.0934065934065933], [1.0879120879120878], [1.0934065934065935], [1.120879120879121], [1.1483516483516483], [1.1263736263736264], [1.120879120879121], [1.1263736263736264], [1.1428571428571428], [1.0604395604395604], [1.0274725274725274], [1.0769230769230769], [1.0769230769230769], [1.302197802197802], [1.2142857142857142], [1.1978021978021975], [1.412087912087912], [1.3681318681318682], [1.3186813186813187], [1.2252747252747254], [1.2252747252747254], [1.0769230769230766], [1.2307692307692308], [1.2362637362637363], [1.1758241758241756], [1.0769230769230769], [1.1868131868131868], [1.1813186813186816], [1.2417582417582418], [1.1263736263736261], [1.1263736263736264], [1.010989010989011], [1.1043956043956045], [1.208791208791209], [1.291208791208791], [1.0714285714285714], [1.2747252747252749], [1.3131868131868134], [1.1043956043956045], [1.1703296703296702], [1.2197802197802199], [1.1428571428571428], [1.1373626373626373], [1.3241758241758244], [1.214285714285714], [1.1593406593406592], [1.2802197802197801], [1.0824175824175823], [1.0384615384615383], [1.1758241758241759], [1.1593406593406592], [1.0769230769230769], [1.1758241758241756], [1.1703296703296706], [1.225274725274725], [1.225274725274725], [1.1043956043956045], [1.1758241758241756], [1.1428571428571426], [1.120879120879121], [1.1978021978021975], [1.2252747252747251], [1.186813186813187], [1.0824175824175826], [1.1868131868131868], [1.3351648351648353], [1.2417582417582418], [1.2582417582417582], [1.1043956043956042], [1.2802197802197801], [1.2692307692307692], [1.1703296703296702], [1.1648351648351647], [1.1043956043956045], [1.1703296703296702], [1.087912087912088], [1.1428571428571428], [1.1043956043956045], [1.2472527472527473], [1.2747252747252746], [1.2362637362637363], [1.120879120879121], [1.2417582417582418], [1.2252747252747254], [1.2142857142857142], [1.2692307692307692], [1.3461538461538463], [1.0934065934065933], [1.1538461538461537], [1.2802197802197801], [1.1593406593406592], [1.1813186813186811], [1.2197802197802199], [1.1538461538461537], [1.10989010989011], [1.120879120879121], [1.2197802197802197], [1.120879120879121], [1.1263736263736264], [1.2637362637362637], [1.1373626373626373], [1.1153846153846154], [1.1428571428571428], [1.0439560439560438], [1.3076923076923075], [1.0054945054945055], [1.0604395604395604], [1.1373626373626373], [1.2417582417582418], [1.2087912087912085], [1.1483516483516483], [1.0494505494505493], [1.2197802197802197], [1.3351648351648353], [1.175824175824176], [1.1318681318681318], [1.2417582417582418], [1.3021978021978022], [1.2747252747252749], [1.2637362637362637], [1.054945054945055], [1.0604395604395604], [1.0549450549450547], [1.1318681318681318], [1.2032967032967035], [1.1428571428571428], [1.1648351648351647], [1.120879120879121], [1.10989010989011], [1.1868131868131866], [1.2527472527472527], [1.192307692307692], [1.197802197802198], [1.1813186813186811], [1.263736263736264], [1.2252747252747254], [1.2582417582417582], [1.3296703296703294], [1.3846153846153846], [1.3186813186813189], [1.3186813186813189], [1.2637362637362637], [1.2967032967032968], [1.3461538461538463], [1.2252747252747251], [1.3186813186813189], [1.4615384615384615], [1.4285714285714284], [1.467032967032967], [1.3296703296703298], [1.5439560439560438], [1.5934065934065933], [1.521978021978022], [1.4670329670329672], [1.4560439560439562], [1.521978021978022], [1.0934065934065933], [1.3626373626373627], [1.1483516483516485], [1.2032967032967035], [1.208791208791209], [1.3406593406593408], [1.3681318681318682], [1.1538461538461537], [1.3516483516483517], [1.2967032967032968], [1.3296703296703294], [1.2912087912087915], [1.3571428571428572], [1.5439560439560438], [1.39010989010989], [1.3461538461538463], [1.5054945054945055], [1.423076923076923], [1.5494505494505495], [1.5054945054945055], [1.521978021978022], [1.6813186813186816], [1.5604395604395604], [1.489010989010989], [1.2582417582417584], [1.2362637362637363], [1.1923076923076923], [1.4175824175824174], [1.2857142857142856], [1.2472527472527473], [1.186813186813187], [1.2362637362637363], [1.3516483516483517], [1.2857142857142856], [1.4065934065934067], [1.4615384615384617], [1.4835164835164836], [1.4670329670329672], [1.291208791208791], [1.5], [1.467032967032967], [1.4560439560439562], [1.4835164835164836], [1.598901098901099], [1.587912087912088], [1.4450549450549453], [1.3956043956043955], [1.5989010989010988], [1.2307692307692308], [1.2197802197802194], [1.1813186813186813], [1.2252747252747254], [1.2967032967032968], [1.4560439560439562], [1.4230769230769231], [1.2637362637362637], [1.3571428571428572], [1.2362637362637363], [1.3516483516483517], [1.3241758241758241], [1.3076923076923077], [1.3736263736263736], [1.5494505494505495], [1.543956043956044], [1.5329670329670328], [1.4120879120879122], [1.6263736263736264], [1.4505494505494507], [1.6538461538461537], [1.5384615384615383], [1.4065934065934065], [1.3681318681318682], [1.3241758241758241], [1.2692307692307692], [1.3076923076923077], [1.2362637362637363], [1.3516483516483517], [1.2912087912087913], [1.3626373626373625], [1.3076923076923077], [1.4010989010989012], [1.2637362637362637], [1.4175824175824174], [1.39010989010989], [1.4175824175824177], [1.521978021978022], [1.5164835164835164], [1.5659340659340661], [1.3846153846153846], [1.2967032967032965], [1.4340659340659339], [1.521978021978022], [1.4670329670329672], [1.4340659340659343], [1.510989010989011], [1.6428571428571428], [1.2747252747252746], [1.379120879120879], [1.2857142857142856], [1.2252747252747251], [1.3131868131868134], [1.5274725274725274], [1.3956043956043955], [1.379120879120879], [1.434065934065934], [1.1703296703296702], [1.3846153846153846], [1.3681318681318682], [1.4340659340659343], [1.5934065934065935], [1.4615384615384617], [1.4505494505494507], [1.587912087912088], [1.5439560439560438], [1.4450549450549453], [1.3681318681318682], [1.478021978021978], [1.543956043956044], [1.4230769230769231], [1.4835164835164836], [1.065934065934066], [1.1868131868131868], [1.2307692307692306], [1.3351648351648353], [1.2857142857142858], [1.2692307692307692], [1.3571428571428572], [1.3186813186813184], [1.5054945054945055], [1.2857142857142856], [1.2582417582417582], [1.39010989010989], [1.4175824175824174], [1.5549450549450547], [1.4175824175824177], [1.5], [1.4065934065934067], [1.5439560439560438], [1.4505494505494505], [1.4835164835164836], [1.6263736263736264], [1.4065934065934067], [1.4505494505494505], [1.5054945054945055], [1.2087912087912087], [1.2472527472527473], [1.3571428571428572], [1.2307692307692306], [1.2692307692307692], [1.2417582417582418], [1.2967032967032965], [1.3461538461538463], [1.3516483516483517], [1.3021978021978025], [1.302197802197802], [1.4670329670329672], [1.4560439560439558], [1.554945054945055], [1.653846153846154], [1.4670329670329672], [1.4945054945054945], [1.4780219780219779], [1.5439560439560438], [1.478021978021978], [1.6483516483516483], [1.60989010989011], [1.5714285714285714], [1.5054945054945055], [1.2472527472527473], [1.1318681318681318], [1.3516483516483517], [1.2912087912087913], [1.302197802197802], [1.4395604395604396], [1.2307692307692308], [1.2857142857142856], [1.4230769230769234], [1.2912087912087913], [1.3736263736263736], [1.3351648351648353], [1.3461538461538463], [1.2472527472527473], [1.5054945054945055], [1.6318681318681318], [1.489010989010989], [1.4560439560439562], [1.5384615384615383], [1.4285714285714286], [1.6758241758241756], [1.4725274725274726], [1.5384615384615383], [1.5384615384615383], [1.1813186813186811], [1.2417582417582418], [1.3021978021978022], [1.39010989010989], [1.3076923076923075], [1.2967032967032968], [1.3131868131868134], [1.3681318681318682], [1.263736263736264], [1.2967032967032968], [1.3296703296703298], [1.3186813186813184], [1.6263736263736264], [1.4670329670329672], [1.3956043956043955], [1.4395604395604396], [1.489010989010989], [1.4725274725274726], [1.3626373626373627], [1.5164835164835164], [1.6153846153846154], [1.6538461538461537], [1.5164835164835164], [1.456043956043956], [1.054945054945055], [1.0329670329670328], [1.2472527472527473], [1.0879120879120878], [1.1263736263736264], [1.1318681318681318], [1.2637362637362637], [1.10989010989011], [1.291208791208791], [1.192307692307692], [1.0494505494505495], [1.1648351648351647], [1.1098901098901095], [1.0934065934065935], [1.120879120879121], [1.0549450549450547], [1.0989010989010988], [1.1593406593406592], [1.214285714285714], [1.1868131868131866], [1.098901098901099], [1.1428571428571428], [1.0549450549450547], [1.1263736263736264], [1.1593406593406592], [1.1263736263736264], [1.0549450549450547], [1.1043956043956042], [1.1043956043956042], [1.0824175824175823], [1.1098901098901097], [1.1483516483516483], [1.0604395604395604], [1.225274725274725], [1.0604395604395607], [1.098901098901099], [1.0769230769230769], [1.214285714285714], [1.1593406593406592], [1.120879120879121], [1.1318681318681318], [1.0384615384615383], [1.0164835164835164], [1.0714285714285712], [1.098901098901099], [1.1813186813186813], [1.1703296703296702], [1.1153846153846154], [1.1483516483516483], [1.1648351648351647], [1.1153846153846154], [1.2527472527472527], [1.0824175824175823], [1.1428571428571428], [1.098901098901099], [1.2692307692307694], [1.1923076923076923], [1.0769230769230769], [1.1813186813186811], [1.0714285714285716], [1.0659340659340661], [1.098901098901099], [1.0164835164835164], [1.120879120879121], [1.2472527472527473], [1.1153846153846154], [1.1043956043956045], [1.1153846153846154], [1.1593406593406592], [1.1868131868131866], [1.10989010989011], [1.2912087912087913], [1.0494505494505495], [1.0824175824175823], [1.1813186813186813], [1.2637362637362637], [1.0439560439560438], [1.10989010989011], [1.1263736263736264], [1.120879120879121], [1.1538461538461537], [1.1263736263736264], [1.1593406593406592], [1.1483516483516483], [1.120879120879121], [1.1703296703296702], [1.1263736263736264], [1.010989010989011], [1.0714285714285714], [1.0879120879120878], [1.1428571428571428], [1.192307692307692], [1.0769230769230769], [1.1538461538461537], [1.0439560439560438], [1.0164835164835164], [1.076923076923077], [1.1428571428571428], [1.208791208791209], [1.065934065934066], [1.0714285714285716], [1.120879120879121], [1.0769230769230769], [1.2032967032967032], [1.1593406593406594], [1.1538461538461537], [1.1758241758241756], [1.2527472527472527], [1.0714285714285714], [1.175824175824176], [1.1428571428571428], [1.1153846153846154], [1.0549450549450547], [1.0604395604395604], [1.2252747252747254], [1.1593406593406594], [1.0439560439560438], [1.021978021978022], [1.1318681318681318], [1.0549450549450547], [1.1538461538461537], [1.0824175824175823], [1.1703296703296702], [1.1648351648351647], [1.1593406593406592], [1.0604395604395602], [1.1373626373626373], [1.1813186813186811], [1.1373626373626373], [1.087912087912088], [1.1978021978021975], [1.0934065934065933], [1.021978021978022], [1.043956043956044], [1.054945054945055], [1.1813186813186811], [1.2142857142857144], [1.10989010989011], [1.192307692307692], [1.098901098901099], [1.0989010989010988], [1.0164835164835164], [1.1978021978021978], [1.120879120879121], [1.0934065934065935], [1.2032967032967032], [1.054945054945055], [1.1758241758241756], [1.1648351648351647], [1.225274725274725], [1.120879120879121], [1.0659340659340657], [1.1373626373626373], [1.0824175824175826], [1.0934065934065935], [1.098901098901099], [1.1373626373626373], [1.1318681318681318], [0.9945054945054945], [1.1648351648351647], [1.1483516483516483], [1.10989010989011], [1.1703296703296702], [1.2307692307692308], [1.1868131868131866], [1.0494505494505493], [1.1373626373626373], [1.0824175824175823], [1.1043956043956042], [1.2417582417582418], [1.1263736263736264], [1.087912087912088], [1.1593406593406592], [1.1483516483516483], [1.197802197802198], [1.1098901098901097], [1.1428571428571428], [1.1263736263736264], [1.0824175824175826], [1.1978021978021975], [1.1318681318681318], [1.1263736263736266], [1.10989010989011], [1.1483516483516483], [1.0934065934065935], [1.1043956043956045], [1.1813186813186811], [1.1318681318681318], [1.0714285714285714], [1.065934065934066], [1.1538461538461537], [0.9945054945054945], [1.1318681318681318], [1.1538461538461537], [1.1978021978021975], [1.0934065934065935], [1.1318681318681318], [1.1263736263736264], [1.0934065934065935], [1.0604395604395604], [1.120879120879121], [1.120879120879121], [1.1098901098901097], [1.0934065934065933], [1.1813186813186811], [1.1593406593406594], [1.0769230769230769], [1.1153846153846154], [1.0769230769230769], [1.186813186813187], [1.0934065934065935], [1.1538461538461537], [1.1813186813186811], [1.0329670329670328], [1.1318681318681318], [1.1153846153846154], [1.1483516483516483], [1.098901098901099], [1.1483516483516483], [1.175824175824176], [1.120879120879121], [1.1538461538461537], [1.1593406593406592], [1.208791208791209], [1.0769230769230769], [1.1153846153846154], [1.2252747252747251], [1.208791208791209], [1.1373626373626373], [1.1978021978021978], [1.0879120879120878], [1.291208791208791], [1.2472527472527473], [1.076923076923077], [1.0879120879120878], [1.1153846153846154], [1.1483516483516483], [1.043956043956044], [1.1703296703296702], [1.0934065934065935], [1.3241758241758244], [1.4285714285714284], [1.3571428571428572], [1.4120879120879122], [1.4065934065934067], [1.3681318681318682], [1.412087912087912], [1.401098901098901], [1.620879120879121], [1.4010989010989015], [1.2527472527472527], [1.4395604395604398], [1.2912087912087913], [1.39010989010989], [1.5824175824175826], [1.5], [1.6483516483516483], [1.5659340659340657], [1.521978021978022], [1.5], [1.576923076923077], [1.6153846153846154], [1.6263736263736264], [1.4670329670329672], [1.291208791208791], [1.1648351648351647], [1.4285714285714288], [1.4945054945054945], [1.5329670329670328], [1.4120879120879122], [1.642857142857143], [1.5714285714285712], [1.3406593406593408], [1.1483516483516483], [1.554945054945055], [1.4615384615384617], [1.3846153846153846], [1.4230769230769231], [1.5054945054945055], [1.4285714285714284], [1.521978021978022], [1.5054945054945055], [1.5604395604395602], [1.4670329670329672], [1.6868131868131866], [1.5714285714285712], [1.5109890109890112], [1.4945054945054945], [1.4615384615384617], [1.3956043956043958], [1.3736263736263736], [1.4835164835164836], [1.3571428571428572], [1.5054945054945055], [1.5934065934065935], [1.3186813186813187], [1.5879120879120878], [1.510989010989011], [1.4670329670329672], [1.3186813186813187], [1.2692307692307692], [1.4835164835164836], [1.5329670329670328], [1.5714285714285714], [1.4065934065934067], [1.5329670329670328], [1.445054945054945], [1.510989010989011], [1.697802197802198], [1.510989010989011], [1.3901098901098903], [1.5824175824175823], [1.3516483516483517], [1.4065934065934067], [1.2582417582417582], [1.4835164835164836], [1.4230769230769231], [1.3956043956043955], [1.4395604395604398], [1.5769230769230769], [1.598901098901099], [1.291208791208791], [1.5054945054945055], [1.3626373626373627], [1.3626373626373627], [1.3736263736263736], [1.3681318681318682], [1.5549450549450547], [1.5054945054945055], [1.5329670329670328], [1.6318681318681318], [1.401098901098901], [1.5494505494505493], [1.4285714285714286], [1.5], [1.708791208791209], [1.335164835164835], [1.4175824175824174], [1.6263736263736264], [1.3736263736263736], [1.39010989010989], [1.4285714285714288], [1.2087912087912085], [1.478021978021978], [1.510989010989011], [1.4010989010989012], [1.4560439560439562], [1.5824175824175823], [1.3681318681318682], [1.6428571428571428], [1.2692307692307692], [1.510989010989011], [1.4560439560439562], [1.5054945054945055], [1.5274725274725274], [1.5054945054945055], [1.6373626373626373], [1.3736263736263736], [1.5494505494505493], [1.5054945054945055], [1.4505494505494505], [1.4175824175824177], [1.5164835164835164], [1.5934065934065935], [1.478021978021978], [1.434065934065934], [1.406593406593407], [1.5054945054945057], [1.401098901098901], [1.3956043956043955], [1.39010989010989], [1.697802197802198], [1.554945054945055], [1.434065934065934], [1.4175824175824174], [1.489010989010989], [1.263736263736264], [1.5329670329670328], [1.5439560439560438], [1.565934065934066], [1.5329670329670328], [1.478021978021978], [1.565934065934066], [1.5494505494505493], [1.4175824175824177], [1.2417582417582418], [1.2912087912087913], [1.5714285714285714], [1.3351648351648353], [1.4175824175824179], [1.3681318681318682], [1.3021978021978022], [1.5769230769230769], [1.4065934065934067], [1.4175824175824174], [1.4120879120879122], [1.4065934065934065], [1.5549450549450547], [1.6043956043956045], [1.510989010989011], [1.467032967032967], [1.5934065934065935], [1.5384615384615385], [1.532967032967033], [1.5439560439560442], [1.6758241758241756], [1.5824175824175823], [1.5934065934065935], [1.2362637362637363], [1.4505494505494505], [1.3681318681318682], [1.4285714285714286], [1.489010989010989], [1.3681318681318682], [1.3461538461538463], [1.5], [1.5384615384615383], [1.4670329670329672], [1.3296703296703298], [1.4505494505494505], [1.5274725274725274], [1.5329670329670328], [1.4725274725274726], [1.3736263736263736], [1.4835164835164836], [1.4395604395604398], [1.4065934065934065], [1.620879120879121], [1.6098901098901097], [1.478021978021978], [1.4120879120879122], [1.5494505494505493], [1.3846153846153846], [1.5439560439560438], [1.3681318681318682], [1.401098901098901], [1.4890109890109893], [1.5494505494505495], [1.456043956043956], [1.4945054945054945], [1.3461538461538463], [1.2307692307692306], [1.4175824175824177], [1.4835164835164836], [1.3956043956043958], [1.576923076923077], [1.456043956043956], [1.5549450549450552], [1.620879120879121], [1.489010989010989], [1.4945054945054945], [1.6153846153846154], [1.521978021978022], [1.5054945054945055], [1.6043956043956045], [1.5494505494505495], [1.3626373626373627], [1.3076923076923077], [1.4505494505494505], [1.5494505494505495], [1.478021978021978], [1.478021978021978], [1.5604395604395604], [1.4230769230769231], [1.4505494505494505], [1.4560439560439562], [1.4835164835164836], [1.3186813186813189], [1.6318681318681318], [1.5494505494505493], [1.6428571428571428], [1.4395604395604396], [1.708791208791209], [1.4560439560439562], [1.5934065934065933], [1.4395604395604396], [1.6428571428571428], [1.5659340659340657], [1.6208791208791207], [1.521978021978022]]
tab_labels = [{'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.001, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.01, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'adaptive', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (2, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (6, 32), 'max_iter': 400}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 300}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 350}, {'activation': 'tanh', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'hidden_layer_sizes': (4, 64), 'max_iter': 400}]

'''
min_value = min([x[0] for x in tab_values])
print(min_value)

min_index = min(range(len(tab_values)), key=lambda i: tab_values[i][0])
print(min_index)
'''
"""
setValue = 1.05
lowEnoughVal = ([])
lowEnoughLabel = ([])
for (x,label) in zip(tab_values, tab_labels):
    if x[0] < setValue:
        lowEnoughVal.append(x)
        lowEnoughLabel.append(label)

for (val, label) in zip(lowEnoughVal, lowEnoughLabel):
    print(f"Value: {val} for {label}")"""


In [42]:
'''import matplotlib.pyplot as plt
import matplotlib.cm as cm
from collections import Counter # Import Counter for easily counting occurrences
from matplotlib.colors import Normalize # For normalizing counts to colormap range


minimal = y_test.min() if y_test.min() < y_pred.min() else y_pred.min()
maximal = y_test.max() if y_test.max() > y_pred.max() else y_pred.max()

x = range(minimal,maximal+1,1)
y = range(minimal,maximal+1,1)

data_points = list(zip(y_test, y_pred))

point_counts = Counter(data_points)

unique_points = list(point_counts.keys())
counts = np.array(list(point_counts.values()))

min_count = counts.min()
max_count = counts.max()

#('viridis', 'plasma', 'hot', 'magma', 'cool')
cmap = cm.plasma

norm = Normalize(vmin=min_count, vmax=max_count if max_count > min_count else min_count + 1)


plt.figure(figsize=(10, 8)) 

for (test_val, pred_val) in unique_points:
    count = point_counts[(test_val, pred_val)]
    color = cmap(norm(count))
    plt.scatter(test_val, pred_val, color=color, s=50, alpha=0.7)

plt.plot(x, y, color='gray', alpha=0.6, linestyle='--', label='Idealne przewidywanie')

plt.xlabel("Odpowiedź człowieka")
plt.ylabel("Przewidziana odpowiedź przez algorytm")

sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label('Liczba punktów o tej samej pozycji')


chosenRating_name = chosenRating.replace("rating_", "")
plt.title(f"Przewidywanie oceny ankietowanego ( {chosenRating_name} ) wybranych filmów z top250 filmów IMDb")
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()
'''

'import matplotlib.pyplot as plt\nimport matplotlib.cm as cm\nfrom collections import Counter # Import Counter for easily counting occurrences\nfrom matplotlib.colors import Normalize # For normalizing counts to colormap range\n\n\nminimal = y_test.min() if y_test.min() < y_pred.min() else y_pred.min()\nmaximal = y_test.max() if y_test.max() > y_pred.max() else y_pred.max()\n\nx = range(minimal,maximal+1,1)\ny = range(minimal,maximal+1,1)\n\ndata_points = list(zip(y_test, y_pred))\n\npoint_counts = Counter(data_points)\n\nunique_points = list(point_counts.keys())\ncounts = np.array(list(point_counts.values()))\n\nmin_count = counts.min()\nmax_count = counts.max()\n\n#(\'viridis\', \'plasma\', \'hot\', \'magma\', \'cool\')\ncmap = cm.plasma\n\nnorm = Normalize(vmin=min_count, vmax=max_count if max_count > min_count else min_count + 1)\n\n\nplt.figure(figsize=(10, 8)) \n\nfor (test_val, pred_val) in unique_points:\n    count = point_counts[(test_val, pred_val)]\n    color = cmap(norm(cou